In [2]:
from gensim.models import Word2Vec
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier

from bs4 import BeautifulSoup
import re, time

from nltk.corpus import stopwords
import nltk.data

%matplotlib inline

In [3]:
model = Word2Vec.load("300features_40minwords_10context")
model

In [4]:
# Word2Vec 모델은 각 단어에 대한 feature 벡터로 구성되며,
# 'syn0'라는 numpy배열로 저장된다.
# syn0의 행 수는 모델 어휘의 갯수
# 컬럼 수는 part2에서 설정한 feature 벡터의 크기(300)
type(model.wv.syn0)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  """


numpy.ndarray

In [5]:
model.wv.syn0.shape

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  """Entry point for launching an IPython kernel.


(11986, 300)

In [6]:
model.wv['flower'][:10]

array([ 0.00938735, -0.03342411, -0.00440804,  0.03798687, -0.05661752,
        0.02456402,  0.07103297, -0.0915316 , -0.07032849,  0.0316743 ],
      dtype=float32)

In [7]:
# 클러스터의 크기 'k'를 어휘 크기의 1/5 이나 평균 5단어로 설정한다. 
word_vectors = model.wv.syn0
num_clusters = word_vectors.shape[0] / 5
num_clusters = int(num_clusters)

# k-means를 정의하고 학습시킨다.
kmeans_clustering = KMeans(n_clusters=num_clusters)
idx = kmeans_clustering.fit_predict(word_vectors)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\pairwise.py:257: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)


In [8]:
idx

array([ 938, 1138, 1770, ...,    9,  240, 1024])

In [17]:
len(model.wv.index2word)

11986

In [19]:
# 각 단어를 클러스터 번호에 매핑되게 word/Index 사전을 만든다.
idx = list(idx)
names = model.wv.index2word
word_centroid_map = {names[i]: idx[i] for i in range(len(names))}

# 1번째 클러스터의 첫 10개 출력
for cluster in range(0,10):
    # 클러스터 번호 출력
    print("\nCluster {}".format(cluster))
    
    # 클러스터 번호와 클러스터에 있는 단어를 찍는다.
    words = []
    for i in range(0, len(list(word_centroid_map.values()))):
        if (list(word_centroid_map.values())[i] == cluster) : 
            words.append(list(word_centroid_map.keys())[i])
    print(words)


Cluster 0
['dumbfound']

Cluster 1
['spotlight', 'banner', 'pike', 'orvill']

Cluster 2
['nurtur', 'humil']

Cluster 3
['paint', 'shadow', 'mirror', 'mist', 'shadowi', 'silhouett', 'foreground', 'autumn']

Cluster 4
['life', 'world', 'youth', 'lifestyl', 'mindset']

Cluster 5
['lucill', 'debra', 'kristin', 'lindsey', 'delilah', 'celest']

Cluster 6
['apollo', 'nasa', 'eject', 'transmiss']

Cluster 7
['choke', 'gnaw']

Cluster 8
['bell', 'justin', 'heather', 'tara', 'paig', 'chloe', 'ursula', 'zoe', 'tori', 'alexandra', 'kristen', 'claudia', 'lucinda', 'janean']

Cluster 9
['claud', 'luc', 'chabrol', 'besson', 'fran', 'francoi', 'lelouch', 'oi', 'petulia']


In [9]:
train = pd.read_csv("../../labeledTrainData.tsv", delimiter='\t', quoting=3, header=0)
test = pd.read_csv("../../testData.tsv", delimiter='\t', quoting=3, header=0)

In [10]:
train.tail()

,id,sentiment,review
24995,"""3453_3""",0,"""It seems like more consideration has gone int..."
24996,"""5064_1""",0,"""I don't believe they made this film. Complete..."
24997,"""10905_3""",0,"""Guy is a loser. Can't get girls, needs to bui..."
24998,"""10194_3""",0,"""This 30 minute documentary Buñuel made in the..."
24999,"""8478_8""",1,"""I saw this movie as a child and it broke my h..."


In [12]:
from KaggleWord2VecUtility import KaggleWord2VecUtility

clean_train_reviews = []
for review in train['review']:
    clean_train_reviews.append(
    KaggleWord2VecUtility.review_to_wordlist(review, remove_stopwords=True))

In [13]:
clean_test_reviews = []
for review in test['review']:
    clean_test_reviews.append(
    KaggleWord2VecUtility.review_to_wordlist(review, remove_stopwords=True))

In [16]:
# bags of centroids 생성
# 속도를 위해 centroid 학습 세트 bag을 미리 할당 (np.zeros)
train_centroids = np.zeros((train['review'].size, num_clusters), dtype='float32')
train_centroids[:5]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [17]:
# centroid는 두 클러스터의 중심점을 정의한 다음, 중심점의 거리를 측정한 것
def create_bags_of_centroids(wordlist, word_centroid_map):
    # 클러스터의 수는 word / centroid map에서 가장 높은 클러스터 인덱스와 같다.
    num_centroids = max(word_centroid_map.values()) + 1
    
    # 속도를 위해 bag of centroids vector를 미리 할당한다.
    bag_of_centroids = np.zeros(num_centroids, dtype='float32')
    
    # loop를 돌며 단어가 word_cetroid_map에 있다면,
    # 해당되는 클러스터의 수를 하나씩 증가시켜준다.
    for word in wordlist:
        if word in word_centroid_map:
            index = word_centroid_map[word]
            bag_of_centroids[index] += 1
    
    return bag_of_centroids

In [21]:
# 학습 리뷰를 bags of centroids로 변환해보자
counter = 0
for review in clean_train_reviews:
    train_centroids[counter] = create_bags_of_centroids(review, word_centroid_map)
    counter += 1
    
# test 리뷰도 같은 방법으로 반복해보자.
test_centroids = np.zeros((test['review'].size, num_clusters), dtype='float32')
counter = 0
for review in clean_test_reviews:
    test_centroids[counter] = create_bags_of_centroids(review, word_centroid_map)
    counter += 1

# 랜덤 포레스트로 학습시켜 예측해보자
forest = RandomForestClassifier(n_estimators=100).fit(train_centroids, train['sentiment'])

In [22]:
from sklearn.model_selection import cross_val_score
score = np.mean(cross_val_score(forest, train_centroids, train['sentiment'], cv=5, scoring='roc_auc'))

In [23]:
score

0.914584048

In [24]:
result = forest.predict(test_centroids )

In [25]:
output = pd.DataFrame(data={"id":test['id'], "sentiment":result})
output.to_csv("./submit_BagsOfCentroids_{0:.5f}.csv".format(score), index=False, quoting=3)